In [1]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = { 'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11 }

In [2]:
class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        return f"{self.rank} of {self.suit}"

In [3]:
class Deck:
    
    def __init__(self):
        self.deck = []
        
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit, rank))
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def __str__(self):
        deck_comp = ''
        
        for card in self.deck:
            deck_comp += '\n' + card.__str__()
        return "The deck has: " + deck_comp
    
    def deal(self):
        return self.deck.pop()

In [4]:
print(Deck())

The deck has: 
Two of Hearts
Three of Hearts
Four of Hearts
Five of Hearts
Six of Hearts
Seven of Hearts
Eight of Hearts
Nine of Hearts
Ten of Hearts
Jack of Hearts
Queen of Hearts
King of Hearts
Ace of Hearts
Two of Diamonds
Three of Diamonds
Four of Diamonds
Five of Diamonds
Six of Diamonds
Seven of Diamonds
Eight of Diamonds
Nine of Diamonds
Ten of Diamonds
Jack of Diamonds
Queen of Diamonds
King of Diamonds
Ace of Diamonds
Two of Spades
Three of Spades
Four of Spades
Five of Spades
Six of Spades
Seven of Spades
Eight of Spades
Nine of Spades
Ten of Spades
Jack of Spades
Queen of Spades
King of Spades
Ace of Spades
Two of Clubs
Three of Clubs
Four of Clubs
Five of Clubs
Six of Clubs
Seven of Clubs
Eight of Clubs
Nine of Clubs
Ten of Clubs
Jack of Clubs
Queen of Clubs
King of Clubs
Ace of Clubs


In [5]:
# Either dealer or player
class Hand: 
    
    def __init__(self):
        self.cards = []
        self.value = 0 # keep track of the current hand total (value over 21 is a losing hand)
        self.aces = 0 # keep track of aces
        
    def add_card(self, card):
        # card passed in from Deck.deal() --> single Card(suit, rank)
        self.cards.append(card)
        self.value += values[card.rank]
        
        # track aces
        if card.rank == "Ace":
            self.aces += 1
    
    def adjust_for_ace(self):
        # if total value > 21 and I still have an ace
        # change my ace to a 1 instead of an 11
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [6]:
class Chips:
    
    def __init__(self, total = 100):
        self.total = total
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

In [7]:
def take_bet(chips):
    
    while True:
        try:
            chips.bet = int(input("How many chips would you like to bet? "))
        except ValueError:
            print("Please provide a number.")
        else:
            if chips.bet > chips.total:
                print("You don't have enough chips. You have {} chips.".format(chips.total))
            else:
                break

In [8]:
def hit(deck, hand):
    hand.add_card(deck.deal())
    hand.adjust_for_ace()

In [9]:
def hit_or_stand(deck, hand):
    global playing # to control an upcoming while loop
    
    while True:
        x = input("\nHit or Stand (h/s)? ").lower()
        
        if x[0] == 'h':
            hit(deck, hand)
        elif x[0] == 's':
            print("Player stands. It's now the dealer's turn.")
            playing = False
        else:
            print("I did't understand your input. Please enter h or s.")
            continue
        break

In [10]:
# display one of the dealer's cards and all of the player's cards
def show_some(player, dealer):
    # show only one of the dealer's cards
    print("\nDealer's Hand (One card hidden!): ")
    print(dealer.cards[1])
    
    # show all (2 cards) of the player's cards
    print("\nPlayer's hand: ", *player.cards, sep="\n")


# display all of the dealer's and player's cards
def show_all(player, dealer):
    # show all of the dealer's cards
    print("\nDealer's hand: ", *dealer.cards, sep="\n")
    
    # display the value of dealer's hand
    print(f"Value of dealer's hand is {dealer.value} ")
    
    # show all of the player's cards
    print("\nPlayer's hand: ", *player.cards, sep="\n")
    
    # display the value of player's hand
    print(f"Value of player's hand is {player.value} ")

In [11]:
def player_busts(player, dealer, chips):
    print("Player Busts!")
    chips.lose_bet()

def player_wins(player, dealer, chips):
    print("Player Wins!")
    chips.win_bet()
    
def dealer_busts(player, dealer, chips):
    print("Dealer Busts!")
    chips.win_bet()

def dealer_wins(player, dealer, chips):
    print("Dealer Wins!")
    chips.lose_bet()

def push(player, dealer):
    print("Dealer and Player tie! PUSH")

In [ ]:
# game
playing = True

while True:
    print("Welcome to Black Jack.")
    
    # create and shuffle deck
    deck = Deck()
    deck.shuffle()
    
    # deal two cards to both the player and dealer
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # set up the player's chips
    player_chips = Chips()
    
    # prompt player for his/her bet
    take_bet(player_chips)
    
    # show only one of the dealer's cards and all of the player's cards
    show_some(player_hand, dealer_hand)
    
    # while player keeps hitting and hasn't exceeded 21
    while playing:
        # prompt player to hit or stand
        hit_or_stand(deck, player_hand)
        
        # show only one of the dealer's cards and all of the player's cards
        show_some(player_hand, dealer_hand)
        
        # if player's hand exceeds 21, run player_busts(), then break out of the loop
        if player_hand.value > 21:
            player_busts(player_hand, dealer_hand, player_chips)
            break
        
    # when player stands and hasn't busted
    if player_hand.value <= 21:
        # if dealer's hand total is under 17, keep hitting
        while dealer_hand.value < 17:
            hit(deck, dealer_hand)
        
        # show all cards
        show_all(player_hand, dealer_hand)
        # run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand, dealer_hand, player_chips)
        else:
            push(player_hand, dealer_hand)
    # inform player of their chips total
    print(f"\nPlayer total chips: {player_chips.total}")
        
    # ask to play again
    new_game = input("Would you like to play another hand (y/n)? ")
        
    if new_game[0].lower() == 'y':
        playing = True
        continue
    else:
        print("Thank you for playing.")
        break

Welcome to Black Jack.
